In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import os
from pprint import pprint

import warnings # supress warnings
warnings.filterwarnings('ignore')

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

In [2]:
def get_soup(url):
    
    try:
        response = requests.get(url)
    except Exception as e:
        print(e)
        
    if not response.status_code == 200:
        print(f"La requête n°{i} a échoué avec le code de statut :", response.status_code)
    if response:
        soup = BeautifulSoup(response.text, 'html.parser')
    return soup
    
#Recherche de la dernière page du site
def get_last_page(url):
    
    soup = get_soup(url)

    # Trouver l'élément <a> avec la classe "page-link"
    a_element = soup.find_all('a', class_='nojs', attrs={"aria-label": "Next"})

    # Extraire la valeur de l'attribut "rel"
    rel_value = list(a_element)[1]['rel']

    # Convertir la valeur en int
    rel_int = int(rel_value[0])

    # Imprimer la valeur en int
    return(rel_int)


def get_pc_dic(div):
    if div.find('a', class_="btn btn-md btn-squared btn-success price go"):
        title = div.find('a', class_='white').text
        price = float(div.find('a', class_="btn btn-md btn-squared btn-success price go").text.replace(' ','').replace('€','').replace(',','.'))
        img_url = div.find('img')['src']
        seller = div.find_all('img')[1]['alt']
        pc_url = "https://www.comparez-malin.fr" + div.find('a', class_='white')['href']

        components = div.find('ul').find_all('li')

        if len(components) == 8:
            Screen = np.nan
            Res = np.nan
            Eth = components[6].text.strip().replace('\xa0','').replace('x\n','')
            Wifi = components[7].text.strip().replace('\xa0','').replace('x\n','')
            
        elif len(components) == 10:
            Screen = components[0].text.strip().replace('\xa0','').replace('x\n','')
            Res = components[1].text.strip().replace('\xa0','').replace('x\n','')
            components = components[2:]
            Eth = components[6].text.strip().replace('\xa0','').replace('x\n','')
            Wifi = components[7].text.strip().replace('\xa0','').replace('x\n','')
            
        elif len(components) == 9:
            Screen = components[0].text.strip().replace('\xa0','').replace('x\n','')
            Res = components[1].text.strip().replace('\xa0','').replace('x\n','')
            components = components[2:]
            Eth = np.nan
            Wifi = components[6].text.strip().replace('\xa0','').replace('x\n','')
            
        elif len(components) == 7:
            Screen = np.nan
            Res = np.nan
            Eth = components[6].text.strip().replace('\xa0','').replace('x\n','')
            Wifi = np.nan
            

        CPU = components[0].text.strip().replace('\xa0','').replace('x\n','')
        GPU = components[1].text.strip().replace('\xa0','').replace('x\n','')
        RAM = components[2].text.strip().replace('\xa0','').replace('x\n','')
        Memory = components[3].text.strip().replace('\xa0','').replace('x\n','')
        OS = components[4].text.strip().replace('\xa0','').replace('x\n','')
        USB = int(components[5].text.strip().replace('\xa0','').replace('x\n','').replace(" x USB",""))


        dic = {"Screen":Screen,
          "Res":Res,
          "CPU":CPU,
          "RAM":RAM,
          "Memory":Memory,
          "OS":OS,
          "USB":USB,
          "Eth":Eth,
          "Wifi":Wifi,
          "Title":title,
          "Price":price,
          "Img_url":img_url,
          "Seller":seller,
          "PC_url":pc_url}

        return(dic)
    else:
        return None

    
#permet de transformer le div initial en dictionnaire avec les informations
def get_portable_dic(div):
    if div.find('a', class_="btn btn-md btn-squared btn-success price go"):
        name = div.find('a', class_='white').text
        price = float(div.find('a', class_="btn btn-md btn-squared btn-success price go").text.replace(' ','').replace('€','').replace(',','.'))
        img_url = div.find('img')['src']
        seller = div.find_all('img')[1]['alt']
        pc_url = "https://www.comparez-malin.fr" + div.find('a', class_='white')['href']

        components = div.find('ul').find_all('li')
            
        
        Screen = components[0].text.strip().replace('\xa0','').replace('x\n','')
        Res = components[1].text.strip().replace('\xa0','').replace('x\n','')
        OS = components[2].text.strip().replace('\xa0','').replace('x\n','')
        CPU = components[3].text.strip().replace('\xa0','').replace('x\n','')
        GPU = components[4].text.strip().replace('\xa0','').replace('x\n','')
        RAM = components[5].text.strip().replace('\xa0','').replace('x\n','')
        Memory = components[6].text.strip().replace('\xa0','').replace('x\n','')
        Wifi = components[7].text.strip().replace('\xa0','').replace('x\n','')


        dic = {"Name":name,
            "Screen":Screen,
            "Res":Res,
            "Price":price,
            "GPU":GPU,
            "CPU":CPU,
            "RAM":RAM,
            "Memory":Memory,
            "OS":OS,
            "Wifi":Wifi,
            "Seller":seller,
            "PC_url":pc_url,
            "Img_url":img_url,
}

        return(dic)
    else:
        return None

#Permet d'obtenir un DataFrame d'information sur chaque pc
def get_pc_df():
    n_page = get_last_page("https://www.comparez-malin.fr/informatique/pc-portable")
    dic_list = []

    for i in range(1, n_page+1):
        
        if i%50 == 0:
            print(i)

        url = "https://www.comparez-malin.fr/informatique/pc-portable/?page=" + str(i)

        soup = get_soup(url)
        
        if soup:
            divs = soup.find_all('div', class_='col-xs-12 col-sm-12 col-md-12 col-lg-6 col-xl-4 col-xxl-4 m-b-10')

            for div in divs : 
                if get_portable_dic(div):
                    dic_list.append(get_portable_dic(div))

    pc_df = pd.DataFrame(dic_list)
    
    return pc_df


def save_df(df, path='pc_dataframe'):
    name = str(datetime.datetime.now().date())+'.csv'
    df.to_csv(path + '/' + name)
    return


def update_df():
    df = get_pc_df()
    save_df(df)
    return

    
def update_bench():
    CPU_df = get_bench_df('https://www.cpubenchmark.net/high_end_cpus.html')
    CPU_pow_df = get_bench_df('https://www.cpubenchmark.net/power_performance.html')
    GPU_df = get_bench_df('https://www.videocardbenchmark.net/high_end_gpus.html')
    
    save_df(CPU_df,'CPU_bench')
    save_df(GPU_df,'GPU_bench')
    return

    
def list_files(path):
    files = []
    for filename in os.listdir(path):
        if os.path.isfile(os.path.join(path, filename)):
            filename = datetime.datetime.strptime(filename.replace('.csv',''), "%Y-%m-%d")
            files.append(filename)
    return files


def load_last_df(path='pc_dataframe'):
    
    f_list = list_files(path)
    
    last_file = sorted(f_list)[-1]
    
    df = pd.read_csv(path + '/'+str(last_file.date())+'.csv', index_col = 0)

    return df

def get_bench_df(url):
    soup = get_soup(url)
    
    url = 'https://' + url.split('/')[2]
    
    
    li_tags = soup.find('ul', class_='chartlist').find_all('li')

    # Parcourir chaque balise 'li' et extraire les informations nécessaires

    dic_list = []
    for li_tag in li_tags:
        prdname = li_tag.find('span', class_='prdname').get_text()
        count = li_tag.find('span', class_='count').get_text()
        price_neww = li_tag.find('span', class_='price-neww').get_text()
        href = li_tag.find('a')['href']

        dic = {'Name':prdname,
              'Note':int(count.replace(',','')),
              'Price ($)':float(price_neww.replace(',','').replace('*','').replace('$','').replace('NA','0')),
              'URL':url+'/'+href}

        dic_list.append(dic)

    df = pd.DataFrame(dic_list)
    return df



In [9]:
CPU_df = get_bench_df('https://www.cpubenchmark.net/high_end_cpus.html')
CPU_pow_df = get_bench_df('https://www.cpubenchmark.net/power_performance.html')
GPU_df = get_bench_df('https://www.videocardbenchmark.net/high_end_gpus.html')



In [11]:
CPU_pow_df

,Name,Note,Price ($),URL
0,Intel Core i7-1260U,1523,0.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-1260U&id=4999
1,Intel Core i7-1250U,1370,426.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-1250U&id=4977
2,AMD Ryzen 7 PRO 7730U,1350,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+PRO+7730U&id=5078
3,Intel Core i9-13900T,1277,549.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i9-13900T&id=5154
4,AMD Ryzen 7 7730U,1262,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+7730U&id=5215
...,...,...,...,...
195,Intel Pentium Silver N6005 @ 2.00GHz,538,0.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Pentium+Silver+N6005+%40+2.00GHz&id=4565
196,AMD Ryzen 5 5600GE,535,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+5+5600GE&id=4390
197,AMD Ryzen 7 7840HS,535,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+7840HS&id=5258
198,AMD Ryzen 7 7700,534,319.79,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+7700&id=5169


In [12]:
#pd.merge(df, GPU_df, left_on='GPU_c', right_on='Name', suffixes=('', '_GPU')).drop(['Name_GPU', 'Price ($)', 'URL', 'GPU_c'], axis=1)
pd.merge(CPU_df, CPU_pow_df, left_on='Name', right_on='Name')

,Name,Note_x,Price ($)_x,URL_x,Note_y,Price ($)_y,URL_y
0,AMD Ryzen 9 7945HX3D,57174,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+7945HX3D&id=5799,1040,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+7945HX3D&id=5799
1,AMD Ryzen 9 7945HX,55269,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+7945HX&id=5232,1005,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+7945HX&id=5232
2,Intel Core i9-13900F,51994,526.03,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i9-13900F&id=5143,800,526.03,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i9-13900F&id=5143
3,AMD Ryzen 9 7900,49069,419.99,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+7900&id=5167,755,419.99,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+7900&id=5167
4,AMD Ryzen 9 PRO 7945,48835,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+PRO+7945&id=5505,751,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+9+PRO+7945&id=5505
...,...,...,...,...,...,...,...
172,Intel Core i5-1035G7 @ 1.20GHz,8304,320.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i5-1035G7+%40+1.20GHz&id=3582,554,320.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i5-1035G7+%40+1.20GHz&id=3582
173,Intel Core i3-1315UE,8294,312.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i3-1315UE&id=5762,553,312.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i3-1315UE&id=5762
174,Intel Core i7-10810U @ 1.10GHz,8276,443.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-10810U+%40+1.10GHz&id=3741,552,443.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-10810U+%40+1.10GHz&id=3741
175,Intel Core i7-1180G7 @ 1.30GHz,8270,426.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-1180G7+%40+1.30GHz&id=4123,551,426.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-1180G7+%40+1.30GHz&id=4123


In [3]:
#update_df()
#update_bench()

### Cleaning and extracting datas

In [4]:
def split_res(res):
    end = res.index('pix')
    chaine = res[:res.index('pix')].replace('Résolution ', "")
    match = re.search(r'x .', chaine)
    if match:
        index = match.start() - 5
        partie_avant_nombre = chaine[:index]
        partie_apres_nombre = chaine[index:]
        return partie_avant_nombre.strip(), partie_apres_nombre.strip()
    else:
        return chaine, ''
    
def split_ram(ram):
    pattern = r'( Go | - | MHz)'
    ram_split = re.split(pattern, ram)
    return [int(ram_split[0]),ram_split[2],int(ram_split[4])]

def split_mem(mem):
    pattern = r'( Go | To | \+ )'
    
    return re.split(pattern, mem)

def get_mem(mem):
    
    if mem[1] == ' To ':
        mem[0] *= 1000
    if mem[5] == ' To ':
        mem[4] *= 1000
    mem[0] += mem[4]
    
    return mem

def df_exp(df):
    df[['Res_qual','Res_px']] = pd.DataFrame(list(df['Res'].apply(split_res)))

    df[['RAM_Go','RAM_tech','RAM_frq']] = pd.DataFrame(list(df['RAM'].apply(split_ram)))
    
    m_list = pd.Series(df['Memory'])

    df_split = pd.DataFrame(list(m_list.apply(split_mem)))
    df_split[0] = df_split[0].astype('int')
    df_split[4] = df_split[4].fillna(0).astype(int)
    df_split = df_split.apply(get_mem, axis=1)[[0,2]]
    df_split[2] = df_split[2].str.replace('(','').str.replace(')','')
    df[['Mem_Go','Mem_tech']] = df_split
    
    df['GPU_brand'] = df['GPU'].str.split(' ', expand=True).iloc[:,0]
    
    return df


def GPU_clean(df):
    
    
    df['GPU_c'] = df['GPU'].str.split(' ', expand=True).iloc[:,1:].fillna('').agg(' '.join, axis=1)

    df['GPU_c'] = df['GPU_c'].str.replace('GeForce','')
    df.GPU_c = df['GPU_c'].str.split('Version', expand=True).iloc[:,0]
    
    
    df['GPU_c'] = df.GPU_c.str.strip()
    
    cleaning_dic = {
        'GTX 1650 Max-Q(4 Go)':'GeForce GTX 1060 with Max-Q Design',
        'GTX 1650 Ti(4 Go)':'GeForce GTX 1650 Ti',
        'GTX 1650 Ti Max-Q(4 Go)':'GeForce GTX 1650 Ti with Max-Q Design',
        'GTX 1650(4 Go)':'GeForce GTX 1650 (Mobile)',
        'RTX 3050(4 Go)': 'GeForce RTX 3050',
        'RTX 3050 Ti Max-Q(4 Go)': 'GeForce RTX 3050 OEM',
        'RTX 3050(6 Go)': 'GeForce RTX 3050 6GB Laptop GPU',
        'RTX 3050 Max-Q(4 Go)': 'GeForce RTX 3050 Laptop GPU',
        'RTX 3050 Ti(4 Go)': 'GeForce RTX 3050 Ti Laptop GPU',
        'RTX 3060 Max-P(6 Go)' : 'GeForce RTX 3060 Laptop GPU', 
        'RTX 3060 Max-Q(6 Go)' : 'GeForce RTX 3060 Laptop GPU', 
        'RTX 3060 Max-Q(8 Go)' : 'GeForce RTX 3060 Ti',
        'RTX 3070 Max-Q(8 Go)':'GeForce RTX 3070 Laptop GPU',
        'RTX 3070 Max-P(8 Go)':'GeForce RTX 3070 Laptop GPU',
        'RTX 3070 Ti Max-P(8 Go)':'GeForce RTX 3070 Ti Laptop GPU',
        'RTX 3070 Ti Max-Q(8 Go)':'GeForce RTX 3070 Ti Laptop GPU',
        'RTX 3080(16 Go)':'GeForce RTX 3080 Ti Laptop GPU',
        'RTX 3080 Ti Max-Q(16 Go)':'GeForce RTX 3080 Ti Laptop GPU',
        'RTX 3080 Max-P(8 Go)':'GeForce RTX 3080 Laptop GPU',
        'RTX 3080 Max-Q(8 Go)':'GeForce RTX 3080 Laptop GPU',
        'RTX 3080 Ti Max-P(16 Go)':'GeForce RTX 3080 Ti Laptop GPU',
        'RTX 3080 Ti Max-P(8 Go)':'GeForce RTX 3080 Ti Laptop GPU',
        'RTX 3080 Max-P(16 Go)':'GeForce RTX 3080 Ti Laptop GPU',
        'RTX 3080 Max-Q(16 Go)':'GeForce RTX 3080 Ti Laptop GPU',
        'RTX 4050 (Mobile)(6 Go)':'GeForce RTX 4050 Laptop GPU',
        'RTX 4050 (Mobile)(8 Go)':'GeForce RTX 4050 Laptop GPU',
        'RTX 4050 (Mobile)(4 Go)':'GeForce RTX 4050 Laptop GPU',
        'RTX 4060 (Mobile)(8 Go)':'GeForce RTX 4060 Laptop GPU',
        'RTX 4060(8 Go)':'GeForce RTX 4060 Laptop GPU', 
        'RTX 4060 (Mobile)(6 Go)':'GeForce RTX 4060 Laptop GPU',
        'RTX 4070 (Mobile)(8 Go)':'GeForce RTX 4070 Laptop GPU',
        'RTX 4080 (Mobile)(12 Go)':'GeForce RTX 4080 Laptop GPU',
        'RTX 4090 (Mobile)(16 Go)':'GeForce RTX 4090 Laptop GPU',
        'Radeon RX 6650M(8 Go)':'Radeon RX 6650M',
        'Radeon RX 6700S(8 Go)':'Radeon RX 6700S',
        'Radeon RX 6500M(4 Go)':'Radeon RX 6650M',
        'Radeon RX 6800M(12 Go)':'Radeon RX 6800M',
        'Radeon RX 6600M(8 Go)':'Radeon RX 6600M',
        'Radeon RX 6850M XT(12 Go)':'Radeon RX 6850M XT',
        'RTX 2050(4 Go)': 'GeForce RTX 2050',
        'RTX A500(4 Go)':'RTX A500 Laptop GPU',
        'RTX A2000(8 Go)':'RTX A2000 8GB Laptop GPU',
        'RTX 3500 ADA(12 Go)':'RTX 3500 Ada Generation Laptop GPU',
        'RTX A2000(4 Go)':'RTX A2000 Laptop GPU',
        'RTX 2000 ADA(8 Go)':'RTX A2000 8GB Laptop GPU',
        'RTX A1000(6 Go)':'RTX A1000 6GB Laptop GPU',
        'RTX A3000(8 Go)':'RTX A3000 Laptop GPU',
        'RTX A4000(12 Go)':'RTX A4000 Laptop GPU',
        'RTX 5000 ADA(16 Go)':'RTX 5000 Ada Generation Laptop GPU',
        'RTX 3000 ADA(8 Go)':'RTX 3000 Ada Generation Laptop GPU',
        'RTX A4500(16 Go)':'RTX A4500 Laptop GPU',
        'RTX A3000(12 Go)':'RTX A3000 12GB Laptop GPU',
        'RTX A5000(16 Go)':'RTX A5000 Laptop GPU',
        'RTX A1000(4 Go)':'RTX A1000 Laptop GPU',
        'RTX A5500(16 Go)':'RTX A5500 Laptop GPU',
        'RTX A3000(6 Go)':'RTX A3000 Laptop GPU',
        'Quadro RTX 5000(16 Go)':'Quadro RTX 5000 (Mobile)',
        'RTX 2070 Super Max-Q(8 Go)':'GeForce RTX 2070 Super with Max-Q Design',
        'RTX 2070 Super(8 Go)':'GeForce RTX 2070 (Mobile)',
        'Quadro RTX 4000(8 Go)':'Quadro RTX 4000 (Mobile)',
        'RTX 2080 Max-Q(8 Go)':'GeForce RTX 2080 with Max-Q Design',
        'Adreno 618':np.nan, 
        'Adreno 690':np.nan, 
        'Adreno 685':np.nan,
        'Arc A370M(4 Go)':'Intel Arc A370M', 
        'Arc A350M(4 Go)':np.nan,
        'Arc A370M(12 Go)':'Intel Arc A370M',
        'Iris Xe Graphics (96 EU) - Raptor Lake':'Intel Iris Xe',
        'Iris Xe Graphics (80 EU) - Raptor Lake':'Intel Iris Xe',
        'Iris Xe Graphics (96 EU)':'Intel Iris Xe',
        'Iris Xe Graphics (80 EU)':'Intel Iris Xe',
        'UHD Graphics (16 EU)':np.nan,
        'UHD Graphics (23 EU)':np.nan,
        'UHD Graphics (24 EU)':np.nan,
        'UHD Graphics (32 EU)':np.nan,
        'UHD Graphics (48 EU) - Alder Lake':np.nan,
        'UHD Graphics (48 EU) - Raptor Lake':np.nan,
        'UHD Graphics (48 EU) - Tiger Lake':np.nan,
        'UHD Graphics (64 EU) - Alder Lake':np.nan,
        'UHD Graphics (64 EU) - Raptor Lake':np.nan,
        'UHD Graphics 600':np.nan,
        'UHD Graphics 605':np.nan,
        'UHD Graphics 610':np.nan,
        'UHD Graphics 620':np.nan,
        'UHD Graphics G1 (32 EU)':np.nan,
        'T1200(4 Go)':'T1200 Laptop GPU',
        'T550(4 Go)':'T550 Laptop GPU',
        'T600(4 Go)':'T600 Laptop GPU',
        'MX350(2 Go)':'GeForce MX350',
        'MX450(2 Go)':'GeForce MX450',
        'MX550(2 Go)':'GeForce MX550',
        'Radeon RX 6650M(4 Go)':'Radeon RX 6650M',
        'Radeon RX 7600S(8 Go)':'Radeon RX 7600S',
        'Quadro P520(4 Go)':np.nan,
        'Quadro T1200(4 Go)':'T1200 Laptop GPU',
        'Radeon 660M':'Radeon 660M Ryzen 5 7535HS',
        'Radeon 680M':'Radeon 680M Ryzen 7 7735HS',
        
    }
    
    df['GPU_c'] = df['GPU_c'].map(cleaning_dic)
    


def CPU_clean(df):
    
    df['CPU_c'] = df['CPU'].str.split('(', expand=True)[0].str.strip()
    
    
    cleaning_dic = {
        'AMD Ryzen 5 7520U(4 coeurs 4.30 GHz)':'AMD Ryzen 5 7520U',
        'AMD Ryzen 7 Pro 6850H(8 coeurs 4.70 GHz)':'AMD Ryzen 7 PRO 6850H',
        'AMD Ryzen 5 4600H(6 coeurs 4.00 GHz)': 'AMD Ryzen 5 4600H',
        'AMD Ryzen 5 7530U(6 coeurs 4.50 GHz)': 'AMD Ryzen 5 7530U',
        'AMD Ryzen 5 5500H(4 coeurs 4.20 GHz)': 'AMD Ryzen 5 5500H',
        'AMD Ryzen 5 5625U(6 coeurs 4.30 GHz)': 'AMD Ryzen 5 5625U',
        'AMD Ryzen 5 5500U(6 coeurs 4.00 GHz)': 'AMD Ryzen 5 5500U',
        'AMD Ryzen 5 7535HS(6 coeurs 4.55 GHz)': 'AMD Ryzen 5 7535HS',
        'AMD Ryzen 5 6600H(6 coeurs 4.50 GHz)': 'AMD Ryzen 5 6600H',
        'AMD Ryzen 5 5600H(6 coeurs 4.10 GHz)': 'AMD Ryzen 5 5600H',
        'AMD Ryzen 5 5600U(6 coeurs 4.20 GHz)': 'AMD Ryzen 5 5600U',
        'AMD Ryzen 5 6600U(6 coeurs 4.50 GHz)': 'AMD Ryzen 5 6600U',
        'AMD Ryzen 5 7535U(6 coeurs 4.55 GHz)': 'AMD Ryzen 5 7535U',
        'AMD Ryzen 5 7520C(4 coeurs 4.30 GHz)': 'AMD Ryzen 5 7520C',
        'AMD Ryzen 5 7640HS(6 coeurs 5.00 GHz)': 'AMD Ryzen 5 7640HS',
        'AMD Ryzen 5 Pro 6650U(6 coeurs 4.50 GHz)': 'AMD Ryzen 5 Pro 6650U',
        'AMD Ryzen 5 4500U(6 coeurs 4.00 GHz)': 'AMD Ryzen 5 4500U',
        'AMD Ryzen 5 3500U(4 coeurs 3.70 GHz)': 'AMD Ryzen 5 3500U',
        'AMD Ryzen 5 Pro 5675U(6 coeurs 4.30 GHz)': 'AMD Ryzen 5 Pro 5675U',
        'AMD Ryzen 5 3450U(4 coeurs 3.50 GHz)': 'AMD Ryzen 5 3450U',
        'AMD Ryzen 5 Pro 5650U(6 coeurs 4.20 GHz)': 'AMD Ryzen 5 Pro 5650U',
        'AMD Ryzen 5 Pro 6650U':'AMD Ryzen 5 PRO 6650U',
        'AMD Ryzen 5 Pro 5675U':'AMD Ryzen 5 PRO 5675U',
        'AMD Ryzen 5 Pro 5650U':'AMD Ryzen 5 PRO 5650U',
        'AMD Ryzen 3 3250C': 'AMD Ryzen 3 3250C',
        'AMD Ryzen 3 3250U': 'AMD Ryzen 3 3250U',
        'AMD Ryzen 7 Pro 6850H': 'AMD Ryzen 7 PRO 6850H',
        'AMD Ryzen 7 Pro 6850U': 'AMD Ryzen 7 PRO 6850U',
        'AMD Ryzen 5 7520C': 'AMD Ryzen 5 7520C',
        'AMD Ryzen 9 7945HX3D': 'AMD Ryzen 9 7945HX3D',
        'AMD Ryzen 7 3700U': 'AMD Ryzen 7 3700U',
        'AMD Ryzen 9 6980HX': 'AMD Ryzen 9 6980HX',
        'AMD Ryzen 5 3500U': 'AMD Ryzen 5 3500U',
        'AMD Ryzen 5 3450U': 'AMD Ryzen 5 3450U',
        'AMD Ryzen 7 4980U': 'AMD Ryzen 7 4980U',
    }
    
    df['CPU_c'] = df['CPU_c'].map(cleaning_dic).fillna(df['CPU_c'])
    
    df['CPU_c'] = df['CPU_c'].str.replace('Pro', 'PRO')
    
def clean_df(df):
    GPU_clean(df)
    CPU_clean(df)
    

### Système de classification

In [30]:
def min_max(sr):
    return (sr-sr.min())/(sr.max()-sr.min())

def comp_notation(df, comp):
    return min_max(df.groupby(comp).median()['Price'].sort_values())

def comps_dic(df, comp_list):
    dic = {}
    for comp in comp_list:
        dic[comp] = comp_notation(df, comp)
    return dic

def notation(pc, Comp_notations, factors):
    note = 0
    for fact in factors.index:
        note += Comp_notations[fact][pc[fact]] * factors[fact]
    return note

def df_noted(df, factors):
    
    factors = factors/factors.sum()
    
    Comp_notations = comps_dic(df, list(factors.index))
    df['Note'] = df.apply(lambda x: notation(x, Comp_notations, factors), axis=1)

    df['Adj_Note'] = (df['Note'] / df['Price'])*1000
    
    return df

#permet de noter les composants continus
def note_function(x, alpha, beta):
    return 1/(1+np.exp(-x/alpha+beta))

def df_noted2(df, factors):
    
    factors = factors/factors.sum()
    df['Note_Finale'] = 0
    for factor in factors.index:
        df['Note_Finale'] += df[factor] * factors[factor]
        
    df['Note_diff'] = df['Note_Finale']-df['Price'].apply(simple_model)
        
    
        
def simple_model(x):
    a1, b1 = -0.33146807, 0.00105002
    a2, b2 = 0.31462099, 0.00020849
    a3, b3 = 0.88924409, 1.38216007e-05

    s1 = 800
    s2 = 3000
    
    if x<= s1:
        return a1 + b1*x
    if s2 < x:
        return a3 + b3*x
    else :
        return a2 + b2*x
    


def note_df(df, factors = pd.Series({'Note_GPU':20,
     'Note_CPU':5, 
     'Note_RAM':2, 
     'Note_Mem':2,
     'Note_Res':2})):

    CPU_df = load_last_df('CPU_bench')
    GPU_df = load_last_df('GPU_bench')
    
    clean_df(df)
    df = df_exp(df)
    CPU_df['Name'] = CPU_df['Name'].str.split('@', expand=True)[0].str.strip()

    p1 = pd.merge(df, GPU_df, left_on='GPU_c', right_on='Name', suffixes=('', '_GPU')).drop(['Name_GPU', 'Price ($)', 'URL', 'GPU_c'], axis=1)
    p2 = pd.merge(p1, CPU_df, left_on='CPU_c', right_on='Name', suffixes=('', '_CPU'), how='left').drop(['Name_CPU', 'Price ($)', 'URL', 'CPU_c'], axis=1)

    p2 = p2.rename({'Note':'Note_GPU' }, axis=1)
    p2['Note_GPU'] = min_max(p2['Note_GPU'])
    p2['Note_CPU'] = min_max(p2['Note_CPU'])
    
    p2['Note_Res'] = min_max(p2['Res_px'].str.split(' x ', expand=True).astype(int).product(axis=1).apply(np.log))
    
    p2['Note_Mem'] = p2['Mem_Go'].apply(lambda x:note_function(x,170, 2.5))
    p2['Note_RAM'] = p2['RAM_Go'].apply(lambda x:note_function(x,7, 2.5))
    
    df_noted2(p2, factors)
    
    return p2

def get_pc_noted(factors = pd.Series({'Note_GPU':20,
     'Note_CPU':5, 
     'Note_RAM':2, 
     'Note_Mem':2,
     'Note_Res':2})):
    
    pc_df = load_last_df()
    pc_df = note_df(pc_df, factors)
    
    return pc_df

def get_best_pc(pc_noted):
    return pc_noted[(pc_noted.Note_Finale > 0.5) & (pc_noted.Price <= 1500 | (pc_noted.Note_diff>=0.02)) 
                    & (pc_noted.Note_diff>=0.02)].sort_values('Note_diff', ascending=False)

def computer_check():
    update_df()
    update_bench()
    pc_noted = get_pc_noted()
    return get_best_pc(pc_noted)

In [41]:
pc_noted = get_pc_noted(factors = pd.Series({'Note_GPU':0,
     'Note_CPU':15, 
     'Note_RAM':0, 
     'Note_Mem':0,
     'Note_Res':0}))

In [43]:
pc_noted.sort_values('Note_Finale', ascending = False)[pc_noted.Price <= 1500].iloc[0:10].sort_values('Price')

,Name,Screen,Res,Price,GPU,CPU,RAM,Memory,OS,Wifi,Seller,PC_url,Img_url,Res_qual,Res_px,RAM_Go,RAM_tech,RAM_frq,Mem_Go,Mem_tech,GPU_brand,Note_GPU,Note_CPU,Note_Res,Note_Mem,Note_RAM,Note_Finale,Note_diff
126,Asus VivoBook S1405VA-LY217W,"Écran 14.0"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1200 pixels\n(16/10),1039.99,Intel Iris Xe Graphics (96 EU) - Raptor Lake,Intel Core i9-13900H(14 coeurs 5.40 GHz),16 Go DDR4 - 3200 MHz,1 To (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 5 • Bluetooth 5.1,Fnac,https://www.comparez-malin.fr/informatique/pc-portable/asus-vivobook-s1405va-ly217w.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2023/35172/asus-vivobook-14-s1405-indie-black-1.jpg,Full HD,1920 x 1200,16,DDR4,3200,1000,SSD M.2 PCI-e 3.0,Intel,0.000000,0.434940,0.253194,0.967148,0.446633,0.434940,-0.096509
369,Asus Vivobook 16 S1605VA-MB253W,"Écran 16.0"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1200 pixels\n(16/10),1109.99,Intel Iris Xe Graphics (96 EU) - Raptor Lake,Intel Core i9-13900H(14 coeurs 5.40 GHz),16 Go DDR4 - 3200 MHz,1 To (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 5 • Bluetooth 4.1,Darty,https://www.comparez-malin.fr/informatique/pc-portable/asus-vivobook-16-s1605va-mb253w.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2023/33443/asus-vivobook-16-grey-1.jpg,Full HD,1920 x 1200,16,DDR4,3200,1000,SSD M.2 PCI-e 3.0,Intel,0.000000,0.434940,0.253194,0.967148,0.446633,0.434940,-0.111103
1132,Lenovo IdeaPad Pro 5 16APH8 (83AR002EFR),"Écran 16.0"" LCD - IPS (anti-reflets) 120 Hz",Résolution QHD 2560 x 1600 pixels\n(16/10),1199.00,Nvidia GeForce RTX 3050(6 Go),AMD Ryzen 7 7840HS(8 coeurs 5.10 GHz),32 Go LPDDR5X - 6400 MHz,512 Go (SSD M.2 PCI-e 4.0),Windows 11 Famille,Wi-Fi 6E • Bluetooth 5.1,Boulanger,https://www.comparez-malin.fr/informatique/pc-portable/lenovo-ideapad-pro-5-16aph8-83ar002efr.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/lenovo/2023/35082/lenovo-ideapad-pro-5-16-gen8-arctic-grey-1.jpg,QHD,2560 x 1600,32,LPDDR5X,6400,512,SSD M.2 PCI-e 4.0,Nvidia,0.308644,0.418842,0.563147,0.625220,0.888095,0.418842,-0.145759
354,Asus Vivobook 16 S1605VA-MB158W,"Écran 16.0"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1200 pixels\n(16/10),1199.95,Intel Iris Xe Graphics (96 EU) - Raptor Lake,Intel Core i9-13900H(14 coeurs 5.40 GHz),16 Go DDR4 - 3200 MHz,512 Go (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 5 • Bluetooth 4.1,Materiel.net,https://www.comparez-malin.fr/informatique/pc-portable/asus-vivobook-16-s1605va-mb158w.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2023/33644/asus-vivobook-16-grey-1.jpg,Full HD,1920 x 1200,16,DDR4,3200,512,SSD M.2 PCI-e 3.0,Intel,0.000000,0.434940,0.253194,0.625220,0.446633,0.434940,-0.129859
293,Asus VivoBook 17 N3704VA-AU032W,"Écran 17.3"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1080 pixels\n(16/9),1299.00,Intel Iris Xe Graphics (96 EU) - Raptor Lake,Intel Core i9-13900H(14 coeurs 5.40 GHz),8 Go DDR4 - 3200 MHz,1 To (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 5 • Bluetooth 4.1,Boulanger,https://www.comparez-malin.fr/informatique/pc-portable/asus-vivobook-17-n3704va-au032w.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2023/34199/asus-vivobook-17-x1702-silver-1.jpg,Full HD,1920 x 1080,8,DDR4,3200,1000,SSD M.2 PCI-e 3.0,Intel,0.000000,0.434940,0.196436,0.967148,0.204705,0.434940,-0.150510
2,Asus Vivobook 16 N1600VA-MB363W,"Écran 16.0"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1200 pixels\n(16/10),1299.99,Intel Iris Xe Graphics (96 EU) - Raptor Lake,Intel Core i9-13900H(14 coeurs 5.40 GHz),24 Go DDR4 - 3200 MHz,1 To (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 5 • Bluetooth 5.1,Asus,https://www.comparez-malin.fr/informatique/pc-portable/asus-vivobook-16-n1600va-mb363w.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2024/36048/asus-vivobook-16-grey-1.jpg,Full HD,1920 x 1200,24,DDR4,3200,1000,SSD M.2 PCI-e 3.0,Intel,0.000000,0.434940,0.253194,

In [6]:
CPU_df = load_last_df('CPU_bench')

In [8]:
get_bench_df('https://www.cpubenchmark.net/power_performance.html')

,Name,Note,Price ($),URL
0,Intel Core i7-1260U,1523,0.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-1260U&id=4999
1,Intel Core i7-1250U,1370,426.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i7-1250U&id=4977
2,AMD Ryzen 7 PRO 7730U,1350,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+PRO+7730U&id=5078
3,Intel Core i9-13900T,1277,549.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Core+i9-13900T&id=5154
4,AMD Ryzen 7 7730U,1262,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+7730U&id=5215
...,...,...,...,...
195,Intel Pentium Silver N6005 @ 2.00GHz,538,0.00,https://www.cpubenchmark.net/cpu.php?cpu=Intel+Pentium+Silver+N6005+%40+2.00GHz&id=4565
196,AMD Ryzen 5 5600GE,535,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+5+5600GE&id=4390
197,AMD Ryzen 7 7840HS,535,0.00,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+7840HS&id=5258
198,AMD Ryzen 7 7700,534,319.79,https://www.cpubenchmark.net/cpu.php?cpu=AMD+Ryzen+7+7700&id=5169


In [42]:
computer_check()

KeyboardInterrupt: 

In [15]:
pc_noted = get_pc_noted()
get_best_pc(pc_noted)

,Name,Screen,Res,Price,GPU,CPU,RAM,Memory,OS,Wifi,Seller,PC_url,Img_url,Res_qual,Res_px,RAM_Go,RAM_tech,RAM_frq,Mem_Go,Mem_tech,GPU_brand,Note_GPU,Note_CPU,Note_Res,Note_Mem,Note_RAM,Note_Finale,Note_diff


In [16]:
pc_noted

,Name,Screen,Res,Price,GPU,CPU,RAM,Memory,OS,Wifi,Seller,PC_url,Img_url,Res_qual,Res_px,RAM_Go,RAM_tech,RAM_frq,Mem_Go,Mem_tech,GPU_brand,Note_GPU,Note_CPU,Note_Res,Note_Mem,Note_RAM,Note_Finale,Note_diff
0,Dell Inspiron 14-5430 (Core i5 / 8 Go / 512 Go),"Écran 14.0"" LCD - WVA (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1200 pixels\n(16/10),679.00,Intel Iris Xe Graphics (80 EU) - Raptor Lake,Intel Core i5-1335U(10 coeurs 4.60 GHz),8 Go LPDDR5 - 4800 MHz,512 Go (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 6E • Bluetooth 5.3,Amazon,https://www.comparez-malin.fr/informatique/pc-portable/dell-inspiron-14-5430-core-i5-8-go-512-go.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/dell/2024/36062/dell-inspiron-14-5430-platinum-silver-1.jpg,Full HD,1920 x 1200,8,LPDDR5,4800,512,SSD M.2 PCI-e 3.0,Intel,0.000000,0.171270,0.253194,0.625220,0.204705,0.097503,-0.283993
1,HP 250 G10 - 8A5J2EA,"Écran 15.6"" LCD - TN (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1080 pixels\n(16/9),840.00,Intel Iris Xe Graphics (80 EU) - Raptor Lake,Intel Core i5-1335U(10 coeurs 4.60 GHz),8 Go DDR4 - 3200 MHz,256 Go (SSD M.2 PCI-e 3.0),Windows 11 Pro,Wi-Fi 6 • Bluetooth 5.3,HP France,https://www.comparez-malin.fr/informatique/pc-portable/hp-250-g10-8a5j2ea.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/hp/2024/36057/hp-250-g10-noir-1.jpg,Full HD,1920 x 1080,8,DDR4,3200,256,SSD M.2 PCI-e 3.0,Intel,0.000000,0.171270,0.196436,0.270100,0.204705,0.070930,-0.418823
2,Asus Vivobook 16 N1600VA-MB363W,"Écran 16.0"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1200 pixels\n(16/10),1299.99,Intel Iris Xe Graphics (96 EU) - Raptor Lake,Intel Core i9-13900H(14 coeurs 5.40 GHz),24 Go DDR4 - 3200 MHz,1 To (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 5 • Bluetooth 5.1,Asus,https://www.comparez-malin.fr/informatique/pc-portable/asus-vivobook-16-n1600va-mb363w.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2024/36048/asus-vivobook-16-grey-1.jpg,Full HD,1920 x 1200,24,DDR4,3200,1000,SSD M.2 PCI-e 3.0,Intel,0.000000,0.434940,0.253194,0.967148,0.716785,0.195128,-0.390528
3,HP ProBook 470 G10 (9B9D7EA),"Écran 17.3"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1080 pixels\n(16/9),1301.52,Intel Iris Xe Graphics (80 EU) - Raptor Lake,Intel Core i5-1335U(10 coeurs 4.60 GHz),16 Go DDR4 - 3200 MHz,512 Go (SSD M.2 PCI-e 3.0),Windows 11 Pro,Wi-Fi 6 • Bluetooth 5.3,HP France,https://www.comparez-malin.fr/informatique/pc-portable/hp-probook-470-g10-9b9d7ea.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/hp/2024/36043/hp-probook-470-g10-1.jpg,Full HD,1920 x 1080,16,DDR4,3200,512,SSD M.2 PCI-e 3.0,Intel,0.000000,0.171270,0.196436,0.625220,0.446633,0.109449,-0.476526
4,Acer Aspire 3 A315-59-56DF,"Écran 15.6"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1080 pixels\n(16/9),599.95,Intel Iris Xe Graphics (80 EU),Intel Core i5-1235U(10 coeurs x 4.40 GHz),8 Go DDR4 - 3200 MHz,512 Go (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 5 • Bluetooth 5.0,Materiel.net,https://www.comparez-malin.fr/informatique/pc-portable/acer-aspire-3-a315-59-56df.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/acer/2024/36041/acer-aspire-3-a315-35-argent-1.jpg,Full HD,1920 x 1080,8,DDR4,3200,512,SSD M.2 PCI-e 3.0,Intel,0.000000,0.104515,0.196436,0.625220,0.204705,0.083074,-0.215417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,Asus VivoBook S15 S533EQ-BN182T,"Écran 15.6"" LCD - IPS (anti-reflets) 60 Hz",Résolution Full HD 1920 x 1080 pixels\n(16/9),1033.29,Nvidia GeForce MX350(2 Go),Intel Core i7-1165G7(4 coeurs 4.70 GHz),8 Go DDR4 - 3200 MHz,512 Go (SSD M.2 PCI-e 3.0),Windows 10 Famille,Wi-Fi 6 • Bluetooth 5.0,Topbiz,https://www.comparez-malin.fr/informatique/pc-portable/asus-vivobook-s15-s533eq-bn182t.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2021/25899/asus-vivobook-s15-s533-gris-1.jpg,Full HD,1920 x 1080,8,DDR4,3200,512,SSD M.2 PCI-e 3.0,Nvidi

In [36]:
pc_noted[(pc_noted.Note_Finale > 0.5) & (pc_noted.Note_diff>=0.01) & (pc_noted.Price <= 1500)].sort_values('Note_diff', ascending = False)

,Name,Screen,Res,Price,GPU,CPU,RAM,Memory,OS,Wifi,Seller,PC_url,Img_url,Res_qual,Res_px,RAM_Go,RAM_tech,RAM_frq,Mem_Go,Mem_tech,GPU_brand,Note_GPU,Note_CPU,Note_Res,Note_Mem,Note_RAM,Note_Finale,Note_diff
1218,Asus TUF Gaming A15 TUF507RW-HN071W,"Écran 15.6"" LCD - IPS (anti-reflets) 144 Hz",Résolution Full HD 1920 x 1080 pixels\n(16/9),999.99,Nvidia GeForce RTX 3070 Ti Max-P(8 Go),AMD Ryzen 7 6800H(8 coeurs 4.70 GHz),16 Go DDR5 - 4800 MHz,1 To (SSD M.2 PCI-e 3.0),Windows 11 Famille,Wi-Fi 6 • Bluetooth 5.2,Cdiscount,https://www.comparez-malin.fr/informatique/pc-portable/asus-tuf-gaming-a15-tuf507rw-hn071w.html,https://cdn.comparez-malin.fr/cache/thumbnail/img/asus/2023/35728/asus-tuf-gaming-a15-2022-red-1.jpg,Full HD,1920 x 1080,16,DDR5,4800,1000,SSD M.2 PCI-e 3.0,Nvidia,0.602233,0.311308,0.196436,0.967148,0.446633,0.542633,0.019524


In [37]:
-1.6449*((4*4)**(1/2))+0.5*4

-4.5796